# Deteccao e correcao de valores ausentes

# Passo 1: Importação das bibliotecas necessárias e carga do dataset

In [ ]:
import pandas as pd #Biblioteca para manipulação e análise de dados
import seaborn as sns #Biblioteca de visualização de dados
import os as os #biblioteca para interagir com o sistema operacional
from sklearn.impute import KNNImputer #biblioteca para aprendizado de máquina (IA)
from google.colab import files #biblioteca para interagir com o ambiente Colab

In [ ]:
# Carregar o conjunto de dados Titanic
titanic = sns.load_dataset('titanic')

In [ ]:
# Visualizar as colunas do conjunto de dados (metadados). Importante para a governanca de dados
print("Colunas do conjunto de dados Titanic:")
print(titanic.columns)

Colunas do conjunto de dados Titanic:
Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')


In [ ]:
# Resumo das colunas
print("\nResumo das colunas:")
print(titanic.info())


Resumo das colunas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB
None


In [ ]:
# Descrição de cada coluna. Esta estrutura de dados e um dicionario
column_descriptions = {
    "survived": "Indica se o passageiro sobreviveu (1) ou não (0).",
    "pclass": "Classe do bilhete do passageiro (1 = 1ª classe, 2 = 2ª classe, 3 = 3ª classe).",
    "sex": "Gênero do passageiro (male = masculino, female = feminino).",
    "age": "Idade do passageiro em anos.",
    "sibsp": "Número de irmãos/cônjuges a bordo do Titanic.",
    "parch": "Número de pais/filhos a bordo do Titanic.",
    "fare": "Tarifa paga pelo bilhete.",
    "embarked": "Porto de embarque (C = Cherbourg; Q = Queenstown; S = Southampton).",
    "class": "Classe do bilhete do passageiro em formato categórico (First, Second, Third).",
    "who": "Descrição da pessoa (man = homem, woman = mulher, child = criança).",
    "adult_male": "Indica se a pessoa é um homem adulto (True) ou não (False).",
    "deck": "Deck em que estava a cabine do passageiro (A, B, C, D, E, F, G, U = desconhecido).",
    "embark_town": "Cidade de embarque do passageiro (Cherbourg, Queenstown, Southampton).",
    "alive": "Indica se o passageiro sobreviveu (yes) ou não (no).",
    "alone": "Indica se o passageiro estava sozinho (True) ou não (False)."
}

# Exibir a descrição das colunas
for column, description in column_descriptions.items():
    print(f"{column}: {description}")

survived: Indica se o passageiro sobreviveu (1) ou não (0).
pclass: Classe do bilhete do passageiro (1 = 1ª classe, 2 = 2ª classe, 3 = 3ª classe).
sex: Gênero do passageiro (male = masculino, female = feminino).
age: Idade do passageiro em anos.
sibsp: Número de irmãos/cônjuges a bordo do Titanic.
parch: Número de pais/filhos a bordo do Titanic.
fare: Tarifa paga pelo bilhete.
embarked: Porto de embarque (C = Cherbourg; Q = Queenstown; S = Southampton).
class: Classe do bilhete do passageiro em formato categórico (First, Second, Third).
who: Descrição da pessoa (man = homem, woman = mulher, child = criança).
adult_male: Indica se a pessoa é um homem adulto (True) ou não (False).
deck: Deck em que estava a cabine do passageiro (A, B, C, D, E, F, G, U = desconhecido).
embark_town: Cidade de embarque do passageiro (Cherbourg, Queenstown, Southampton).
alive: Indica se o passageiro sobreviveu (yes) ou não (no).
alone: Indica se o passageiro estava sozinho (True) ou não (False).


In [ ]:
# Visualizar as primeiras linhas do conjunto de dados
print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  


# Passo 2: Download do arquivo original

In [ ]:
# Definir o nome do arquivo
file_name = 'titanic.csv'

In [ ]:
# Exportar o DataFrame para um arquivo CSV
titanic.to_csv(file_name, index=False)

In [ ]:
# Mensagem que o arquivo foi gerado
print(f"Conjunto de dados salvo como '{file_name}'")

Conjunto de dados salvo como 'titanic.csv'


In [ ]:
# Baixar o arquivo CSV para o computador local
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Passo 3: Identificacao dos dados nulos

In [ ]:
# Verificar valores ausentes
print(titanic.isnull().sum())

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [ ]:
# Resumo dos valores ausentes por coluna
missing_values = titanic.isnull().sum()
print(missing_values[missing_values > 0])

age            177
embarked         2
deck           688
embark_town      2
dtype: int64


# Passo 4: Usando um algoritmo de IA para preencher os valores ausentes
O KNNImputer da biblioteca scikit-learn é uma ferramenta utilizada para preencher valores ausentes em um conjunto de dados com base no algoritmo K-Nearest Neighbors (k-NN). O KNNImputer utiliza a média (ou outro critério, dependendo da configuração) dos valores mais próximos para imputar os valores ausentes em cada ponto de dados.

In [ ]:
# Selecionar apenas as colunas numéricas para o KNNImputer
titanic_numeric = titanic.select_dtypes(include=['float64', 'int64'])

In [ ]:
# Inicializar o KNNImputer
imputer = KNNImputer(n_neighbors=5)

In [ ]:
# Aplicar o KNNImputer
titanic_imputed = pd.DataFrame(imputer.fit_transform(titanic_numeric), columns=titanic_numeric.columns)

In [ ]:
# Verificar se os valores ausentes foram preenchidos
print(titanic_imputed.isnull().sum())

survived    0
pclass      0
age         0
sibsp       0
parch       0
fare        0
dtype: int64


In [ ]:
# Comparar os dados originais com valores ausentes preenchidos
print("Valores ausentes antes da imputação:\n", titanic.isnull().sum())
print("Valores ausentes após a imputação:\n", titanic_imputed.isnull().sum()) #somente os valores numericos sao exibidos

Valores ausentes antes da imputação:
 survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64
Valores ausentes após a imputação:
 survived    0
pclass      0
age         0
sibsp       0
parch       0
fare        0
dtype: int64


# Passo 5: Download do arquivo com os valores imputados

In [ ]:
# Definir o nome do arquivo
file_name = 'titanic_imputed.csv'

In [ ]:
# Exportar o DataFrame para um arquivo CSV
titanic_imputed.to_csv(file_name, index=False)

In [ ]:
print(f"Conjunto de dados salvo como '{file_name}'")

Conjunto de dados salvo como 'titanic_imputed.csv'


In [ ]:
# Baixar o arquivo CSV para o computador local
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>